In [1]:
# TMDB Open API 감독, 줄거리, 나라 전처리하는 파일입니다.
import requests
import json

In [2]:
# json 에서 API Key값 가져오기 : key, token
file_path = "./APIKey/TMDB.json"
with open(file_path, 'r') as file:
    api = json.load(file)

In [3]:
key = api['key']
token = api['token']

In [4]:
movies = "616037"
# movie info...
url = f"https://api.themoviedb.org/3/movie/{movies}?api_key={key}"
req = requests.get(url)
api_info = json.loads(req.text)

In [15]:
type(api_info)

dict

In [6]:
# title str값 => id값으로 정수 부여
# genres dict -> key(int 'id') : value(str 'name')
# country : 다른 것도 같이 확인 필요!!!
title = api_info['title']
country = api_info['production_countries'][0]['iso_3166_1']
overview = api_info['overview']

In [36]:
print("title: ", title, ", country: ", country)
print("overview: ", overview)
print("genres: ", api_info['genres'])

title:  Thor: Love and Thunder , country:  US
overview:  After his retirement is interrupted by Gorr the God Butcher, a galactic killer who seeks the extinction of the gods, Thor enlists the help of King Valkyrie, Korg, and ex-girlfriend Jane Foster, who now inexplicably wields Mjolnir as the Mighty Thor. Together they embark upon a harrowing cosmic adventure to uncover the mystery of the God Butcher’s vengeance and stop him before it’s too late.
genres:  [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}]


In [8]:
# cast, crew ...
url = f"https://api.themoviedb.org/3/movie/{movies}/credits?api_key={key}"
req = requests.get(url)
api_people = json.loads(req.text)

In [10]:
for person in api_people['crew']:
    try :  
        if(person['job'] == 'Director'):
            print(person)
    except:
        pass 

{'adult': False, 'gender': 2, 'id': 55934, 'known_for_department': 'Directing', 'name': 'Taika Waititi', 'original_name': 'Taika Waititi', 'popularity': 49.668, 'profile_path': '/zCLBXGo5BS2e27srDBa5WpRnKul.jpg', 'credit_id': '5d2e0eb4caab6d164099c274', 'department': 'Directing', 'job': 'Director'}


In [18]:
# id And title
# total_pages : 34599
# total_data : 691978
page = 1
url = f"https://api.themoviedb.org/3/discover/movie?page={page}&api_key={key}"
req = requests.get(url)
api_movie = json.loads(req.text)

In [22]:
for movie in api_movie['results']:
    print(movie['id'], movie['title'])

616037 Thor: Love and Thunder
438148 Minions: The Rise of Gru
507086 Jurassic World Dominion
361743 Top Gun: Maverick
725201 The Gray Man
718789 Lightyear
453395 Doctor Strange in the Multiverse of Madness
756999 The Black Phone
854467 Indemnity
919355 Dragon Knight
634649 Spider-Man: No Way Home
759175 The Princess
728366 Borrego
924482 The Ledge
961484 Last Seen Alive
629015 Shut In
836225 The Exorcism of God
675353 Sonic the Hedgehog 2
626735 Dog
894169 Vendetta


In [57]:
import numpy as np
from tqdm import tqdm
pages = np.arange(500) + 1

In [60]:
movie_list = []
for page in tqdm(pages):
    url = f"https://api.themoviedb.org/3/discover/movie?page={page}&api_key={key}"
    req = requests.get(url)
    api_movie = json.loads(req.text)
    for movie in api_movie['results']:
        movie_id = []
        movie_id.append(movie['id'])
        for movie in movie_id:
            url = f"https://api.themoviedb.org/3/movie/{movie}?api_key={key}"
            req = requests.get(url)
            api_info = json.loads(req.text)
            title= {movie : api_info['title']}
            budget = api_info['budget']
            genres = api_info["genres"]
            country = api_info['production_countries']
            vote_average = api_info['vote_average']
            url = f"https://api.themoviedb.org/3/movie/{movie}/credits?api_key={key}"
            req = requests.get(url)
            api_people = json.loads(req.text)
            for actor in api_people['cast']:
                try:
                    if(actor['order'] == 1):
                        main_charactor = {actor['id'] : actor['name']}
                except:
                    pass
            for crew in api_people['crew']:
                try :  
                    if(crew['job'] == 'Director'):
                        director = {crew['id'] : crew['name']}
                except:
                    pass 
            movie_list.append([title, budget, genres, country, main_charactor, director, vote_average])
            del title, budget, genres, country, vote_average
            director = "-"
        del movie_id


100%|██████████| 500/500 [46:54<00:00,  5.63s/it] 


In [71]:
import pandas as pd
df = pd.DataFrame(movie_list)
df.to_csv("movie_data", index=False)

In [88]:
movie_df = pd.read_csv("movie_data", names="title, budget, genres, country, main_charactor, director, vote_average".split(", "), skiprows=1)

In [95]:
print("total data: ", len(movie_df))
movie_df.head()

total data:  10000


,title,budget,genres,country,main_charactor,director,vote_average
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.753
1,{438148: 'Minions: The Rise of Gru'},85000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{124747: 'Pierre Coffin'},{8023: 'Kyle Balda'},7.775
2,{507086: 'Jurassic World Dominion'},165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{18997: 'Bryce Dallas Howard'},{930707: 'Colin Trevorrow'},7.110
3,{361743: 'Top Gun: Maverick'},170000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{996701: 'Miles Teller'},{86270: 'Joseph Kosinski'},8.335
4,{718789: 'Lightyear'},200000000,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{74688: 'Keke Palmer'},{7929: 'Angus MacLane'},7.359


In [90]:
# 87개
NoDirector = movie_df[movie_df['director'] == '-']

In [102]:
NoDirector.head(3)

,title,budget,genres,country,main_charactor,director,vote_average
168,{975117: 'Father There Is Only One 3'},0,[],[],{19492: 'Viola Davis'},-,6.8
177,{755433: 'The Monster at the End of This Story'},0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{1213631: 'Leslie Carrara-Rudolph'},-,3.0
205,"{992187: ""Chip and Potato: Chip's Holiday""}",0,"[{'id': 16, 'name': 'Animation'}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",{76700: 'Briana Buckmaster'},-,5.3


In [128]:
movie_df1 = movie_df.drop(NoDirector.index)
len(movie_df1)

9913

In [129]:
NoBudget = movie_df1[movie_df1['budget'] == 0]
len(NoBudget)

In [131]:
movie_df2 = movie_df1.drop(NoBudget.index)
len(movie_df2)

5309

In [143]:
movie_df2 = movie_df2.drop(movie_df2[movie_df2['genres'].apply(lambda x: len(x) == 2)].index)
len(movie_df2)

5307

In [145]:
movie_df2 = movie_df2.drop(movie_df2[movie_df2['country'].apply(lambda x: len(x) == 2)].index)
len(movie_df2)

5302

In [148]:
movie_df2[movie_df2['vote_average'].apply(lambda x: x == 0)].index

Int64Index([ 370,  463,  912,  976, 2845, 4289, 4314, 5369, 5850, 6125, 6570,
            6990, 8128, 8360, 9396],
           dtype='int64')

In [150]:
movie_df2.to_csv("movie_data_final.csv", index=False)

In [164]:
movieDF = movie_df2

In [161]:
#import ast
#test = ast.literal_eval(movie_df2["genres"][0])
#print(test)
#형태가 리스트/딕셔너리인 문자열을 다시 리스트/딕셔너리 형태로 바꿀 수 있다.